**Подключение к гугл диску**

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')
!ln -s /content/gdrive/My\ Drive/ /mydrive
!ls /mydrive

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
'Colab Notebooks'  'My Drive'   yolov3


**1) Клонируем репозиторий с файлами, нужными для обучения**





In [ ]:
!git clone https://github.com/AlexeyAB/darknet

Cloning into 'darknet'...
remote: Enumerating objects: 15316, done.
remote: Counting objects: 100% (4/4), done.
remote: Total 15316 (delta 3), reused 3 (delta 3), pack-reused 15312
Receiving objects: 100% (15316/15316), 13.72 MiB | 19.36 MiB/s, done.
Resolving deltas: 100% (10406/10406), done.


**2) Компиляция Darknet с использованием Nvidia GPU и OpenCV**


In [ ]:

%cd darknet
!sed -i 's/OPENCV=0/OPENCV=1/' Makefile
!sed -i 's/GPU=0/GPU=1/' Makefile
!sed -i 's/CUDNN=0/CUDNN=1/' Makefile
!make

/content/darknet
mkdir -p ./obj/
mkdir -p backup
chmod +x *.sh
g++ -std=c++11 -std=c++11 -Iinclude/ -I3rdparty/stb/include -DOPENCV `pkg-config --cflags opencv4 2> /dev/null || pkg-config --cflags opencv` -DGPU -I/usr/local/cuda/include/ -DCUDNN -Wall -Wfatal-errors -Wno-unused-result -Wno-unknown-pragmas -fPIC -Ofast -DOPENCV -DGPU -DCUDNN -I/usr/local/cudnn/include -c ./src/image_opencv.cpp -o obj/image_opencv.o
./src/image_opencv.cpp: In function ‘void draw_detections_cv_v3(void**, detection*, int, float, char**, image**, int, int)’:
./src/image_opencv.cpp:946:23: warning: variable ‘rgb’ set but not used [-Wunused-but-set-variable]
                 float rgb[3];
                       ^~~
./src/image_opencv.cpp: In function ‘void draw_train_loss(char*, void**, int, float, float, int, int, float, int, char*, float, int, int, double)’:
./src/image_opencv.cpp:1147:13: warning: this ‘if’ clause does not guard... [-Wmisleading-indentation]
             if (iteration_old == 0)
           

**3) Настройка сети для обучения модели YOLOV3**

Берем файл cfg/yolov3.cfg  и копируем в наш репозиторий, где создаем файл cfg/yolov3_training.cfg

In [ ]:
!cp cfg/yolov3.cfg cfg/yolov3_training.cfg

Изменяем необходимые параметры под нашу модель

In [ ]:
!sed -i 's/batch=1/batch=64/' cfg/yolov3_training.cfg
!sed -i 's/subdivisions=1/subdivisions=16/' cfg/yolov3_training.cfg
!sed -i 's/max_batches = 500200/max_batches = 4000/' cfg/yolov3_training.cfg
!sed -i '610 s@classes=80@classes=1@' cfg/yolov3_training.cfg
!sed -i '696 s@classes=80@classes=1@' cfg/yolov3_training.cfg
!sed -i '783 s@classes=80@classes=1@' cfg/yolov3_training.cfg
!sed -i '603 s@filters=255@filters=18@' cfg/yolov3_training.cfg
!sed -i '689 s@filters=255@filters=18@' cfg/yolov3_training.cfg
!sed -i '776 s@filters=255@filters=18@' cfg/yolov3_training.cfg

Создаем папку с данными и записываем туда данные под нашу модель

In [ ]:
!echo "hookah" > data/obj.names
!echo -e 'classes= 1\ntrain  = data/train.txt\nvalid  = data/test.txt\nnames = data/obj.names\nbackup = /mydrive/yolov3' > data/obj.data
!mkdir data/obj

Скачиваем преобученную модель darknet53.conv.74 с весами

In [ ]:

!wget https://pjreddie.com/media/files/darknet53.conv.74

--2021-10-24 12:48:43--  https://pjreddie.com/media/files/darknet53.conv.74
Resolving pjreddie.com (pjreddie.com)... 128.208.4.108
Connecting to pjreddie.com (pjreddie.com)|128.208.4.108|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 162482580 (155M) [application/octet-stream]
Saving to: ‘darknet53.conv.74’

darknet53.conv.74   100%[===================>] 154.96M  20.9MB/s    in 8.1s    

2021-10-24 12:48:52 (19.0 MB/s) - ‘darknet53.conv.74’ saved [162482580/162482580]



**4) Извлекаем наши изображения из архива**



In [ ]:
!unzip /mydrive/yolov3/images.zip -d data/obj

Archive:  /mydrive/yolov3/images.zip
  inflating: data/obj/hookah1.txt    
  inflating: data/obj/hookah2.jpg    
  inflating: data/obj/hookah2.txt    
  inflating: data/obj/hookah3.jpg    
  inflating: data/obj/hookah3.txt    
  inflating: data/obj/hookah4.jpg    
  inflating: data/obj/hookah4.txt    
  inflating: data/obj/hookah5.jpg    
  inflating: data/obj/hookah5.txt    
  inflating: data/obj/hookah6.jpg    
  inflating: data/obj/hookah6.txt    
  inflating: data/obj/hookah7.jpg    
  inflating: data/obj/hookah7.txt    
  inflating: data/obj/hookah8.jpg    
  inflating: data/obj/hookah8.txt    
  inflating: data/obj/hookah9.jpg    
  inflating: data/obj/hookah9.txt    
  inflating: data/obj/hookah10.jpg   
  inflating: data/obj/hookah10.txt   
  inflating: data/obj/hookah11.jpg   
  inflating: data/obj/hookah11.txt   
  inflating: data/obj/hookah12.jpg   
  inflating: data/obj/hookah12.txt   
  inflating: data/obj/hookah13.jpg   
  inflating: data/obj/hookah13.txt   
  inflating: 

In [ ]:
# Изменяем индексы в .txt файлах с 15 на 0, так как у нас всего один класс
# 
import glob
import os
import re

txt_file_paths = glob.glob(r"data/obj/*.txt")
for i, file_path in enumerate(txt_file_paths):
    
    with open(file_path, "r") as f_o:
        lines = f_o.readlines()

        text_converted = []
        for line in lines:
            print(line)
            numbers = re.findall("[0-9.]+", line)
            print(numbers)
            if numbers:

                
                text = "{} {} {} {} {}".format(0, numbers[1], numbers[2], numbers[3], numbers[4])
                text_converted.append(text)
                print(i, file_path)
                print(text)
        
        with open(file_path, 'w') as fp:
            for item in text_converted:
                fp.writelines("%s\n" % item)

15 0.540000 0.455556 0.342222 0.768889

['15', '0.540000', '0.455556', '0.342222', '0.768889']
0 data/obj/hookah158.txt
0 0.540000 0.455556 0.342222 0.768889
15 0.439130 0.434783 0.472464 0.826087

['15', '0.439130', '0.434783', '0.472464', '0.826087']
1 data/obj/hookah124.txt
0 0.439130 0.434783 0.472464 0.826087
15 0.513105 0.495714 0.453629 0.854286

['15', '0.513105', '0.495714', '0.453629', '0.854286']
2 data/obj/hookah123.txt
0 0.513105 0.495714 0.453629 0.854286
15 0.513125 0.493750 0.318750 0.915000

['15', '0.513125', '0.493750', '0.318750', '0.915000']
3 data/obj/hookah196.txt
0 0.513125 0.493750 0.318750 0.915000
15 0.472678 0.476449 0.420765 0.807971

['15', '0.472678', '0.476449', '0.420765', '0.807971']
4 data/obj/hookah284.txt
0 0.472678 0.476449 0.420765 0.807971
15 0.600586 0.502199 0.300781 0.995601

['15', '0.600586', '0.502199', '0.300781', '0.995601']
5 data/obj/hookah276.txt
0 0.600586 0.502199 0.300781 0.995601
15 0.484093 0.497266 0.497349 0.952344

['15', '0.48

In [ ]:
import glob
images_list = glob.glob("data/obj/*.jpg")
print(images_list)

['data/obj/hookah233.jpg', 'data/obj/hookah67.jpg', 'data/obj/hookah256.jpg', 'data/obj/hookah252.jpg', 'data/obj/hookah43.jpg', 'data/obj/hookah103.jpg', 'data/obj/hookah129.jpg', 'data/obj/hookah284.jpg', 'data/obj/hookah280.jpg', 'data/obj/hookah255.jpg', 'data/obj/hookah206.jpg', 'data/obj/hookah50.jpg', 'data/obj/hookah274.jpg', 'data/obj/hookah125.jpg', 'data/obj/hookah191.jpg', 'data/obj/hookah56.jpg', 'data/obj/hookah61.jpg', 'data/obj/hookah145.jpg', 'data/obj/hookah100.jpg', 'data/obj/hookah127.jpg', 'data/obj/hookah279.jpg', 'data/obj/hookah272.jpg', 'data/obj/hookah251.jpg', 'data/obj/hookah221.jpg', 'data/obj/hookah296.jpg', 'data/obj/hookah58.jpg', 'data/obj/hookah6.jpg', 'data/obj/hookah246.jpg', 'data/obj/hookah164.jpg', 'data/obj/hookah106.jpg', 'data/obj/hookah157.jpg', 'data/obj/hookah266.jpg', 'data/obj/hookah248.jpg', 'data/obj/hookah250.jpg', 'data/obj/hookah201.jpg', 'data/obj/hookah86.jpg', 'data/obj/hookah10.jpg', 'data/obj/hookah163.jpg', 'data/obj/hookah118.j

In [ ]:
# Записываем фотографии в файл для тренировочной выборки
file = open("data/train.txt", "w") 
file.write("\n".join(images_list)) 
file.close() 

**6) Начинаем обучение**

In [ ]:

!./darknet detector train data/obj.data cfg/yolov3_training.cfg darknet53.conv.74 -dont_show

Выходные данные были обрезаны до нескольких последних строк (5000).
 total_bbox = 114525, rewritten_bbox = 0.000873 % 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 82 Avg (IOU: 0.819060), count: 4, class_loss = 0.859403, iou_loss = 0.076143, total_loss = 0.935545 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 94 Avg (IOU: 0.000000), count: 1, class_loss = 0.000087, iou_loss = 0.000000, total_loss = 0.000087 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 106 Avg (IOU: 0.000000), count: 1, class_loss = 0.000000, iou_loss = 0.000000, total_loss = 0.000000 
 total_bbox = 114529, rewritten_bbox = 0.000873 % 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 82 Avg (IOU: 0.767153), count: 6, class_loss = 0.534466, iou_loss = 0.336818, total_loss = 0.871284 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 94 Avg (IOU: 0.000000), count: 1, class_loss = 0.000032, iou_loss = 0.000000, total_loss =